## Análise Exploratória de Dados (EDA) - Dataset de Sepsis

**Objetivo da Análise**

* Compreender a estrutura e qualidade dos dados
* Analisar padrões de valores faltantes
* Identificar relações entre variáveis e a ocorrência de sepsis
* Gerar insights para construção de modelos preditivos

**Sobre o Dataset**

* **Problema**: Detecção precoce de sepsis em pacientes de UTI
* **Tipo**: Classificação binária (Sepsis vs Não-Sepsis)
* **Características**: Dados de sinais vitais, exames laboratoriais e informações demográficas


## Acessando o dataset de treino do Drive

Para funcionar no Google Colab, é necessário criar um atalho do diretório MDA no seu próprio Drive e então rodar os dois comandos abaixo e conceder permissão ao seu drive quando rodar a célula logo abaixo.

[Link](https://towardsdatascience.com/simplify-file-sharing-44bde79a8a18/) detalhando como funciona

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
# modificar para o diretorio que contém os dados de teste e treino
%cd /content/drive/MyDrive/MDA/Train\ and\ test\ data\ -\ Proj\ DM/

!ls

Se carregado corretamente o output de

```
!ls
```

 mostrará os arquivos de teste e treino:

dataset_sepsis_test.csv  dataset_sepsis_train.csv

## 1.Importação das Bibliotecas

Primeiro, vamos importar todas as bibliotecas necessárias para nossa análise exploratória:

In [ ]:
# Importação das bibliotecas essenciais para análise exploratória
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

# Configurações para otimizar as visualizações
warnings.filterwarnings('ignore')
plt.style.use('default')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10


print("Bibliotecas importadas com sucesso")

## 2.Carregamento e Descrição da Estrutura e Qualidade do Dataset

Nesta seção, carregamos o dataset de treino e analisamos suas características básicas para compreender a estrutura dos dados e identificar possíveis problemas de qualidade.

In [ ]:
# Carregamento do dataset de treino
train_df = pd.read_csv('dataset_sepsis_train.csv')

# Análise da estrutura básica do dataset
print("INFORMAÇÕES GERAIS DO DATASET\n")
print(f"Dimensões do dataset (shape): {train_df.shape}")
print(f"Número de amostras: {train_df.shape[0]:,}")
print(f"Número de features: {train_df.shape[1]}")
print(f"Tamanho em memória: {train_df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

print(f"\nINFORMAÇÕES DETALHADAS DO DATASET")
print("=" * 50)
# Usar info() para mostrar tipos de dados e valores não-nulos
train_df.info()

print(f"\nESTATÍSTICAS DESCRITIVAS (TRANSPOSTAS)")
print("=" * 50)
# Mostrar estatísticas descritivas transpostas para melhor visualização
display(train_df.describe().T)

# Visualizar as primeiras linhas
print(f"\nPRIMEIRAS 5 LINHAS DO DATASET")
print("=" * 50)
display(train_df.head())

### 2.1 Análise de Valores Faltantes

A análise sistemática de valores faltantes é essencial para identificar padrões de missings e definir estratégias de tratamento adequadas. Esta análise nos permite entender se os dados faltam de forma aleatória ou seguem algum padrão específico.

#### 2.1.1 Análise Verbosa de Valores Faltantes

In [ ]:
# Análise quantitativa dos valores faltantes
missing_count = train_df.isnull().sum()
missing_pct = (missing_count / len(train_df)) * 100

# Criar DataFrame para análise organizada
missing_df = pd.DataFrame({
    'Coluna': missing_count.index,
    'Valores_Faltantes': missing_count.values,
    'Porcentagem': missing_pct.values
}).sort_values('Porcentagem', ascending=False)

# Estatísticas gerais
total_samples = len(train_df)
mean_missing = missing_pct.mean()
vars_with_missing = (missing_count > 0).sum()
total_vars = len(missing_count)

print("ANÁLISE DE VALORES FALTANTES\n")
print(f"Total de amostras: {total_samples:,}")
print(f"Total de variáveis: {total_vars}")
print(f"Variáveis com missing values: {vars_with_missing}")
print(f"Percentual médio de missing values: {mean_missing:.3f}%")

print(f"\nTOP 10 VARIÁVEIS COM MAIS MISSING VALUES:")
top_10_missing = missing_df.head(10)
display(top_10_missing)

# Categorização por nível de missing
very_high = missing_df[missing_df['Porcentagem'] > 50]
high_missing = missing_df[(missing_df['Porcentagem'] > 20) & (missing_df['Porcentagem'] <= 50)]
medium_missing = missing_df[(missing_df['Porcentagem'] > 5) & (missing_df['Porcentagem'] <= 20)]
low_missing = missing_df[(missing_df['Porcentagem'] > 0) & (missing_df['Porcentagem'] <= 5)]

print(f"\nCATEGORIZAÇÃO POR NÍVEL DE MISSING:\n")
print(f"Muito Alto (>50%): {len(very_high)} variáveis")
print(f"Alto (20-50%): {len(high_missing)} variáveis")
print(f"Médio (5-20%): {len(medium_missing)} variáveis")
print(f"Baixo (0-5%): {len(low_missing)} variáveis")
print(f"Sem missing: {len(missing_df[missing_df['Porcentagem'] == 0])} variáveis")

#### 2.1.2 Visualização dos Padrões de Missing Values

A visualização dos padrões de valores faltantes nos ajuda a identificar se existe alguma estrutura nos dados ausentes e a definir estratégias de tratamento mais eficazes.

In [ ]:
# Filtrar apenas variáveis com missing values para visualização
missing_with_nans = missing_df[missing_df['Porcentagem'] > 0]

# Configurar subplots - 2 gráficos principais
fig, axes = plt.subplots(1, 2, figsize=(20, 12))

# 1. Gráfico de barras horizontais - TODAS as colunas com missing values
# Preparar dados para o seaborn barplot
missing_sorted = missing_with_nans.sort_values('Porcentagem', ascending=True)  # Crescente para barras horizontais

# Criar gráfico de barras horizontais usando seaborn
sns.barplot(
    x=missing_sorted['Porcentagem'],
    y=missing_sorted['Coluna'],
    orient='h',
    palette='rocket_r',  # Paleta de cores
    ax=axes[0]
)

# Configurar títulos e labels
axes[0].set_title(f'Percentual de Valores Nulos (NaNs) por Coluna\n({len(missing_sorted)} variáveis com dados ausentes)',
                    fontsize=14, pad=20)
axes[0].set_xlabel('Percentual (%)', fontsize=12)
axes[0].set_ylabel('Colunas do Dataset', fontsize=12)

# Adicionar valores nas barras
for i, (idx, row) in enumerate(missing_sorted.iterrows()):
    pct = row['Porcentagem']
    axes[0].text(pct + 1, i, f'{pct:.1f}%',
                ha='left', va='center', fontsize=9, fontweight='bold')

# Adicionar linhas de referência
axes[0].axvline(20, color='orange', linestyle='--', alpha=0.8, label='20% (Baixo)')
axes[0].axvline(80, color='red', linestyle='--', alpha=0.8, label='80% (Alto)')
axes[0].legend(loc='upper right')

# Configurar grid e limites
axes[0].grid(axis='x', alpha=0.3)
axes[0].set_xlim(0, 105)

# 2. Histograma da distribuição de missing values (mais segmentado)
# Criar bins de 10 em 10
bins = range(0, 101, 10)

axes[1].hist(missing_with_nans['Porcentagem'], bins=bins, alpha=0.7,
            color='#34495e', edgecolor='black', rwidth=0.8)

axes[1].set_xlabel('Percentual de Missing Values (%)', fontsize=12)
axes[1].set_ylabel('Número de Variáveis', fontsize=12)
axes[1].set_title('Distribuição de Missing Values por Faixa', fontsize=14, pad=20)

# Configurar eixo X
axes[1].set_xticks(bins)
axes[1].set_xlim(0, 100)

# Adicionar linha vertical para a média
mean_missing = missing_with_nans['Porcentagem'].mean()
axes[1].axvline(mean_missing, color='red', linestyle='--', linewidth=2,
                label=f'Média: {mean_missing:.1f}%')
axes[1].legend()

# Configurar grid
axes[1].grid(axis='y', alpha=0.3)

# Adicionar anotações no histograma
counts, bins_edges = np.histogram(missing_with_nans['Porcentagem'], bins=bins)
for i, count in enumerate(counts):
    if count > 0:
        bin_center = (bins_edges[i] + bins_edges[i+1]) / 2
        axes[1].text(bin_center, count + max(counts)*0.01, str(count),
                    ha='center', va='bottom', fontweight='bold', fontsize=10)

plt.tight_layout()
plt.show()

print(f"\nEstatísticas descritivas dos missing values:")
print(f"  • Média: {missing_with_nans['Porcentagem'].mean():.1f}%")
print(f"  • Mediana: {missing_with_nans['Porcentagem'].median():.1f}%")
print(f"  • Desvio padrão: {missing_with_nans['Porcentagem'].std():.1f}%")
print(f"  • Mínimo: {missing_with_nans['Porcentagem'].min():.1f}%")
print(f"  • Máximo: {missing_with_nans['Porcentagem'].max():.1f}%")

Estratégias de tratamento possíveis por faixa de valores faltantes

- Muito baixo/Baixo (<20%): Imputação simples (mediana/moda)
- Médio (20-50%): Imputação específica por domínio clínico ()
- Alto (50-80%): Considerar remoção ou imputação avançada
- Muito alto (≥80%): Forte candidato à remoção. Chance de tratamento se estiver associado a muitas instâncias de SepsisLabel=1


#### 2.1.3 Visualização de Outliers e Inconsistências

Baseando-se nas estatísticas descritivas, vamos identificar e visualizar variáveis com potenciais problemas de qualidade dos dados, incluindo outliers extremos e valores inconsistentes.

In [ ]:
# Análise de outliers e inconsistências baseada nas estatísticas descritivas
# Obter estatísticas descritivas apenas para variáveis numéricas (excluindo SepsisLabel)
numerical_vars = train_df.select_dtypes(include=[np.number]).columns
numerical_vars = numerical_vars.drop('SepsisLabel')  # Remover a variável target


# Criar boxplot para visualizar distribuições e outliers
# Como temos muitas variáveis, vamos fazer um gráfico grande com subplots
n_vars = len(numerical_vars)
n_cols = 4  # 4 gráficos por linha
n_rows = (n_vars + n_cols - 1) // n_cols  # Arredondar para cima

fig, axes = plt.subplots(n_rows, n_cols, figsize=(20, n_rows * 4))
axes = axes.flatten() if n_vars > 1 else [axes]

for i, var in enumerate(numerical_vars):
    # Criar boxplot para cada variável
    data = train_df[var].dropna()

    if len(data) > 0:
        axes[i].boxplot(data, patch_artist=True,
                       boxprops=dict(facecolor='lightblue', alpha=0.7),
                       medianprops=dict(color='red', linewidth=2),
                       flierprops=dict(marker='o', markerfacecolor='red', markersize=2, alpha=0.5))

        axes[i].set_title(f'{var}\n(n={len(data):,})', fontsize=10, pad=10)
        axes[i].set_ylabel('Valores')
        axes[i].grid(True, alpha=0.3)

        # Adicionar estatísticas básicas no gráfico
        stats_text = f'Min: {data.min():.2f}\nMédia: {data.mean():.2f}\nMax: {data.max():.2f}'
        axes[i].text(0.02, 0.98, stats_text, transform=axes[i].transAxes,
                    fontsize=8, verticalalignment='top',
                    bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    else:
        axes[i].text(0.5, 0.5, 'Sem dados\nválidos',
                    transform=axes[i].transAxes, ha='center', va='center')
        axes[i].set_title(var, fontsize=10)

# Remover subplots extras se houver
for i in range(n_vars, len(axes)):
    fig.delaxes(axes[i])

plt.tight_layout()
plt.show()

### 2.2 Estrutura Temporal dos Dados

Um aspecto fundamental para compreender este dataset é que **cada linha não representa um paciente único**, mas sim um **momento específico no tempo** para diferentes pacientes. O dataset é uma coleção de séries temporais clínicas onde múltiplas observações são registradas para cada paciente ao longo do tempo.

#### 2.2.1 Análise Verbosa das Frequências de Coleta por Paciente 

In [ ]:
patient_id_cols = [col for col in train_df.columns if 'patient' in col.lower() or 'id' in col.lower()]
print(f"Colunas que podem ser ID do paciente: {patient_id_cols}")

print(f"\nDIMENSÕES DO DATASET:")
print(f"   Total de observações (linhas): {len(train_df):,}")
print(f"   Total de variáveis (colunas): {len(train_df.columns)}")

print(f"\nVARIÁVEIS TEMPORAIS IDENTIFICADAS:")

# Análise da variável Hour
hour_non_null = train_df['Hour'].notna().sum()
hour_unique = train_df['Hour'].nunique()
hour_min = train_df['Hour'].min()
hour_max = train_df['Hour'].max()

print(f"\n   Hour:")
print(f"      Observações válidas: {hour_non_null:,}")
print(f"      Valores únicos: {hour_unique:,}")
print(f"      Range: {hour_min:.2f} - {hour_max:.2f} horas")
print(f"      Interpretação: Horas desde admissão na UTI")
print(f"      Máximo: {hour_max/24:.1f} dias na UTI")

# Análise da variável HospAdmTime
hospadm_non_null = train_df['HospAdmTime'].notna().sum()
hospadm_unique = train_df['HospAdmTime'].nunique()
hospadm_min = train_df['HospAdmTime'].min()
hospadm_max = train_df['HospAdmTime'].max()

print(f"\n   HospAdmTime:")
print(f"      Observações válidas: {hospadm_non_null:,}")
print(f"      Valores únicos: {hospadm_unique:,}")
print(f"      Range: {hospadm_min:.2f} - {hospadm_max:.2f} horas")
print(f"      Interpretação: Tempo de internação hospitalar")
print(f"      Máximo: {hospadm_max:.1f} horas para ser admitido na UTI depois de ser admitido no hospital")

# Estimativa de pacientes únicos usando características demográficas
demographic_cols = ['Age', 'Gender', 'Unit1', 'Unit2']
print(f"\nESTIMATIVA DE PACIENTES ÚNICOS:")
print(f"   (Baseada em combinação de: {', '.join(demographic_cols)})")

demo_combination = train_df[demographic_cols].fillna('Unknown')
demo_string = demo_combination.astype(str).agg('_'.join, axis=1)
unique_combinations = demo_string.nunique()

print(f"   Combinações únicas de características: {unique_combinations:,}")
print(f"   Observações por combinação (média): {len(train_df) / unique_combinations:.1f}")

combo_counts = demo_string.value_counts()
print(f"\n   DISTRIBUIÇÃO DE OBSERVAÇÕES:")
print(f"      Min observações por paciente: {combo_counts.min()}")
print(f"      Max observações por paciente: {combo_counts.max()}")
print(f"      Mediana observações por paciente: {combo_counts.median():.0f}")
print(f"      Média observações por paciente: {combo_counts.mean():.1f}")

# Padrão temporal das observações
print(f"\nPADRÃO TEMPORAL DAS OBSERVAÇÕES:")

hour_dist = train_df['Hour'].value_counts().sort_index()
print(f"   Horas com mais observações:")
top_hours = hour_dist.head(5)
for hour, count in top_hours.items():
    print(f"      Hora {hour:.0f}: {count:,} observações ({count/len(train_df)*100:.1f}%)")

unique_hours = train_df['Hour'].nunique()

print(f"   Frequência média de coleta: {len(train_df) / unique_hours:.1f} pacientes por momento")
print(f"\n   Total de momentos temporais únicos: {unique_hours:,}")

**Implicações para a análise:**

As análises devem considerar a dependência temporal, pois pacientes podem ter múltiplas observações ao longo do tempo e a variável target (SepsisLabel) pode mudar ao longo do tempo para o mesmo paciente

#### 2.2.2 Visualização das Distribuições pela Variável Hour

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# 1. Distribuição das observações por hora
hour_counts = train_df['Hour'].value_counts().sort_index()
axes[0,0].plot(hour_counts.index, hour_counts.values, marker='o', linewidth=2, markersize=4)
axes[0,0].set_title('Distribuição de Observações por Hora', fontsize=12)
axes[0,0].set_xlabel('Horas desde admissão na UTI até a observação de dados clínicos')
axes[0,0].set_ylabel('Número de observações')
axes[0,0].grid(True, alpha=0.3)

# 2. Boxplot da distribuição de horas
axes[0,1].boxplot(train_df['Hour'].dropna(), patch_artist=True,
                 boxprops=dict(facecolor='lightblue', alpha=0.7))
axes[0,1].set_title('Distribuição das Horas de Observação', fontsize=12)
axes[0,1].set_ylabel('Horas')
axes[0,1].grid(True, alpha=0.3)

# 3. Distribuição de SepsisLabel ao longo do tempo
sepsis_by_hour = train_df.groupby('Hour')['SepsisLabel'].agg(['count', 'sum']).reset_index()
sepsis_by_hour['sepsis_rate'] = sepsis_by_hour['sum'] / sepsis_by_hour['count'] * 100

# Filtrar apenas horas com pelo menos 100 observações para estabilidade
stable_hours = sepsis_by_hour[sepsis_by_hour['count'] >= 100]

axes[1,0].plot(stable_hours['Hour'], stable_hours['sepsis_rate'],
              marker='o', linewidth=2, markersize=4, color='red')
axes[1,0].set_title('Taxa de Sepsis ao Longo do Tempo', fontsize=12)
axes[1,0].set_xlabel('Horas desde admissão na UTI até a observação de dados clínicos')
axes[1,0].set_ylabel('Taxa de Sepsis (%)')
axes[1,0].grid(True, alpha=0.3)

# 4. Histograma do número de observações por paciente estimado
demo_combination = train_df[demographic_cols].fillna('Unknown')
demo_string = demo_combination.astype(str).agg('_'.join, axis=1)
combo_counts = demo_string.value_counts()

axes[1,1].hist(combo_counts.values, bins=30, alpha=0.7, color='green', edgecolor='black')
axes[1,1].set_title('Distribuição do Número de Observações\npor Paciente Estimado', fontsize=12)
axes[1,1].set_xlabel('Número de observações por paciente')
axes[1,1].set_ylabel('Frequência')
axes[1,1].grid(True, alpha=0.3)

# Adicionar estatísticas no gráfico
mean_obs = combo_counts.mean()
median_obs = combo_counts.median()
axes[1,1].axvline(mean_obs, color='red', linestyle='--',
                 label=f'Média: {mean_obs:.1f}')
axes[1,1].axvline(median_obs, color='orange', linestyle='--',
                 label=f'Mediana: {median_obs:.0f}')
axes[1,1].legend()

plt.suptitle('Estrutura Temporal do Dataset de Sepsis', fontsize=14, y=0.98)
plt.tight_layout()
plt.show()

#### 2.2.3 Interpretação dos Gráficos sobre a Variável Hour

**1. Distribuição de Observações por Hora (Superior Esquerdo)**

Este gráfico revela um padrão temporal crítico no dataset:

- **Concentração inicial**: Há uma concentração massiva de observações nas primeiras horas após admissão na UTI (0-50 horas)
- **Pico máximo**: Ocorre nas primeiras horas, com mais de 30.000 observações
- **Declínio exponencial**: O número de observações diminui drasticamente após as primeiras 100 horas
- **Cauda longa**: Alguns pacientes permanecem na UTI por até 300+ horas (12+ dias)

**Implicação clínica**: A maioria dos pacientes tem estadia curta na UTI ou são transferidos/recebem alta rapidamente, enquanto uma minoria permanece por períodos prolongados.

**2. Distribuição das Horas de Observação (Superior Direito)**

O boxplot complementa a análise temporal:

- **Mediana baixa**: A mediana está próxima a 25-30 horas, confirmando que metade dos pacientes tem observações concentradas no início
- **Outliers extremos**: Existem casos com observações muito tardias (acima de 250 horas)
- **IQR compacto**: O intervalo interquartil é relativamente pequeno, indicando que 75% das observações ocorrem nas primeiras ~50 horas
- **Assimetria positiva**: A distribuição é fortemente enviesada para a direita

**Interpretação**: O tempo de permanência na UTI segue um padrão típico de cuidados intensivos, com a maioria dos casos sendo resolvidos rapidamente.

**3. Taxa de Sepsis ao Longo do Tempo (Inferior Esquerdo)**

Este é o gráfico mais clinicamente relevante, mostrando a evolução temporal do risco de sepsis:

- **Período inicial de baixo risco**: Nas primeiras 50 horas, a taxa de sepsis permanece baixa (0-5%)
- **Transição crítica**: Entre 50-100 horas, há um aumento gradual mas consistente na taxa de sepsis
- **Pico de risco**: A partir de 100 horas, a taxa de sepsis aumenta significativamente, chegando a 15-20%
- **Padrão progressivo**: O risco continua aumentando com o tempo de permanência

**Significado clínico**:
- Pacientes com internação prolongada têm risco progressivamente maior de desenvolver sepsis
- O tempo de permanência é um **preditor temporal** importante para sepsis
- Após 100 horas na UTI, o risco praticamente triplica

**4. Distribuição do Número de Observações por Paciente Estimado (Inferior Direito)**

Este histograma revela a estrutura longitudinal do dataset:

- **Concentração em baixas contagens**: A maioria dos pacientes tem poucas observações (< 500)
- **Distribuição assimétrica**: Distribuição com cauda longa à direita
- **Mediana vs Média**: Mediana (36) muito menor que a média (83.2), confirmando assimetria
- **Casos extremos**: Alguns pacientes têm mais de 3.000 observações

**Interpretações importantes**:
- **Heterogeneidade temporal**: Pacientes variam drasticamente no tempo de monitoramento
- **Natureza longitudinal**: Confirma que cada paciente contribui com múltiplas observações
- **Viés potencial**: Pacientes mais graves podem ter mais observações (mais tempo na UTI)

#### 2.2.4 Síntese dos Insights sobre as Relações com Hour


**Principais Descobertas:**

1. **Padrão de Permanência na UTI**:
   - 75% dos pacientes têm observações concentradas nas primeiras 75 horas
   - Mediana de permanência: ~30 horas (1,25 dias)
   - Casos extremos podem chegar a 300+ horas (12+ dias)

2. **Relação Temporal com Sepsis**:
   - **Janela de baixo risco**: Primeiras 50 horas (taxa < 5%)
   - **Janela de transição**: 50-100 horas (aumento gradual)
   - **Janela de alto risco**: >100 horas (taxa pode ultrapassar 15%)

3. **Estrutura dos Dados**:
   - Dataset é uma **coleção de séries temporais**, não registros únicos de pacientes
   - Cada linha = um momento temporal para um paciente específico
   - Variabilidade alta no número de observações por paciente (mediana: 36, máximo: >3000)

**Implicações para Modelagem**:

- **Dependência temporal**: Modelos devem considerar a sequência temporal das observações
- **Feature temporal**: O tempo de permanência é um preditor importante
- **Validação temporal**: Necessária validação que preserve a ordem temporal
- **Threshold temporal**: Considerar diferentes estratégias para janelas temporais específicas

**Relevância Clínica**:

- O tempo de permanência prolongado na UTI está associado ao aumento progressivo do risco de sepsis
- Pacientes que permanecem >100 horas na UTI constituem um grupo de alto risco que requer monitoramento intensificado
- A detecção precoce de sepsis é mais crítica nos primeiros dias de internação, quando a taxa é naturalmente baixa

### 2.3 Separação de Features e Target

Separamos a variável alvo (SepsisLabel) das features para facilitar as análises subsequentes. Essa separação é fundamental para a análise univariada e bivariada posterior.

In [ ]:
# Separação entre features (X) e variável target (y)
X_train = train_df.drop('SepsisLabel', axis=1)
y_train = train_df['SepsisLabel']

# Mostrar valores únicos por coluna
print(f"\nANÁLISE DE VALORES ÚNICOS POR COLUNA")
print("=" * 50)
unique_counts = train_df.nunique().sort_values(ascending=False)
print("Valores únicos por coluna:")
display(unique_counts.to_frame('Valores_Únicos'))

print(f"\nColunas com poucos valores únicos (potencialmente categóricas):")
few_unique = unique_counts[unique_counts <= 10]
display(few_unique.to_frame('Valores_Únicos'))

# Mostrar distribuição da variável target
print(f"\nDISTRIBUIÇÃO DA VARIÁVEL TARGET")
print("=" * 50)
target_counts = y_train.value_counts().sort_index()
target_pct = y_train.value_counts(normalize=True).sort_index() * 100

for label, count in target_counts.items():
    pct = target_pct[label]
    status = "Não-Sepsis" if label == 0 else "Sepsis"
    print(f"{status} ({label}): {count:,} amostras ({pct:.2f}%)")

# Calcular razão de desbalanceamento
imbalance_ratio = target_counts[0] / target_counts[1]
print(f"\nRazão de desbalanceamento: {imbalance_ratio:.1f}:1")
print(f"Para cada caso de sepsis, existem {imbalance_ratio:.1f} casos de não-sepsis")

#### 2.3.1 Visualização da Distribuição da Variável Target

In [ ]:
# Visualização da distribuição da variável target
plt.figure(figsize=(15, 5))

# Gráfico 1: Contagem absoluta
plt.subplot(1, 2, 1)
bars = plt.bar(['Não-Sepsis', 'Sepsis'], target_counts.values,
               color=['#3498db', '#e74c3c'], alpha=0.8, edgecolor='black')
plt.title('Distribuição Absoluta das Classes', fontsize=14, pad=20)
plt.xlabel('Classes')
plt.ylabel('Frequência')

# Adicionar valores nas barras
for bar, count in zip(bars, target_counts.values):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + max(target_counts)*0.01,
            f'{count:,}', ha='center', va='bottom', fontweight='bold')

# Gráfico 2: Gráfico de pizza
plt.subplot(1, 2, 2)
wedges, texts, autotexts = plt.pie(target_pct.values,
                                  labels=['Não-Sepsis', 'Sepsis'],
                                  autopct='%1.2f%%',
                                  colors=['#3498db', '#e74c3c'],
                                  startangle=90,
                                  explode=(0, 0.1))
plt.title('Proporção das Classes', fontsize=14, pad=20)

# Melhorar formatação dos rótulos
for autotext in autotexts:
    autotext.set_color('white')
    autotext.set_fontweight('bold')

plt.tight_layout()
plt.show()

Podemos ver que o dataset está altamente desbalanceado. Precisaremos tomar alguns cuidados na fase de avaliação e amostragem:
- Utilizar métricas adequadas (F1-Score, AUC-ROC, Precision-Recall)
- Aplicar técnicas de balanceamento (SMOTE, undersampling)
- Configurar class_weight nos algoritmos de ML
- Considerar threshold optimization

#### 2.3.2 Identificação e Remoção de Linhas duplicadas

In [ ]:
# Verificar se há duplicatas no dataset
duplicates = train_df.duplicated().sum()
print(f"\nLinhas duplicadas: {duplicates}")
if duplicates > 0:
    train_df = train_df.drop_duplicates()
    print(f"Linhas duplicadas removidas. Novo shape do dataset: {train_df.shape}")

## 3.Análise Univariada Detalhada

A análise univariada examina cada variável individualmente para compreender suas distribuições, identificar outliers e avaliar características estatísticas importantes como assimetria e curtose.

### 3.1 Distribuições de Principais Sinais Vitais e Biomarcadores

Análise das distribuições das variáveis clínicas mais importantes para detecção de sepsis, incluindo sinais vitais, parâmetros de oxigenação e biomarcadores laboratoriais.

**Legenda para as Cores de Fundo:**

- Verde claro: Distribuição aproximadamente normal (|skewness| < 0.5)
- Amarelo claro: Moderadamente assimétrica (0.5 ≤ |skewness| ≤ 1.0)
- Vermelho claro: Fortemente assimétrica (|skewness| > 1.0)

In [ ]:
# Análise univariada focada em variáveis clínicas importantes para sepsis

# Definir variáveis clínicas prioritárias
clinical_vars = ['DBP', 'Resp', 'WBC', 'Lactate', 'Creatinine', 'BUN', 
                 'Platelets', 'Glucose', 'SaO2', 'FiO2', 'Age']

# Selecionar 9 para visualização (3x3 grid)
priority_vars = clinical_vars[:9]

# Criar grid 3x3 para histogramas com KDE
fig, axes = plt.subplots(3, 3, figsize=(18, 15))
axes = axes.flatten()

for idx, var in enumerate(priority_vars):
    if idx < len(axes):
        ax = axes[idx]
        
        # Dados válidos (sem NaN)
        data = X_train[var].dropna()
        
        if len(data) > 0:
            # Histograma com KDE
            sns.histplot(data, kde=True, alpha=0.7, color='steelblue', ax=ax)
            
            # Calcular estatísticas
            mean_val = data.mean()
            median_val = data.median()
            skewness = data.skew()
            
            # Adicionar linhas de referência
            ax.axvline(mean_val, color='red', linestyle='--', alpha=0.8, label=f'Média: {mean_val:.1f}')
            ax.axvline(median_val, color='orange', linestyle='--', alpha=0.8, label=f'Mediana: {median_val:.1f}')
            
            # Título e labels
            ax.set_title(f'{var}\nSkewness: {skewness:.2f} | n={len(data):,}', fontsize=12, pad=10)
            ax.set_xlabel(var)
            ax.set_ylabel('Frequência')
            ax.legend(fontsize=8)
            ax.grid(True, alpha=0.3)
            
            # Análise de normalidade visual
            if abs(skewness) > 1:
                skew_interpretation = "Assimétrica"
                ax.set_facecolor('#fff2f2')  # Fundo levemente vermelho para alta assimetria
            elif abs(skewness) < 0.5:
                skew_interpretation = "~Normal"
                ax.set_facecolor('#f2fff2')  # Fundo levemente verde para distribuição normal
            else:
                skew_interpretation = "Mod. Assim."
                ax.set_facecolor('#fffff2')  # Fundo levemente amarelo para assimetria moderada
        
        else:
            ax.text(0.5, 0.5, f'{var}\nSem dados válidos', transform=ax.transAxes, 
                   ha='center', va='center', fontsize=12)
            ax.set_title(var)

# Remover subplots vazios
for idx in range(len(priority_vars), len(axes)):
    fig.delaxes(axes[idx])

plt.suptitle('Distribuições das Principais Variáveis Clínicas\n(Cores de fundo indicam normalidade da distribuição)', 
             fontsize=16, y=0.98)
plt.tight_layout()
plt.show()

print(f"\nAssimetria das {len(priority_vars)} variáveis analisadas:")
for col in priority_vars:
    skew_val = X_train[col].skew()
    if skew_val > 1:
        interpretation = "Fortemente assimétrica à direita"
    elif skew_val < -1:
        interpretation = "Fortemente assimétrica à esquerda"
    elif abs(skew_val) < 0.5:
        interpretation = "Aproximadamente simétrica"
    else:
        interpretation = "Moderadamente assimétrica"

    print(f"  {col}: {skew_val:.2f} ({interpretation})")


**Padrões de Distribuição Observados**

A análise univariada das principais variáveis clínicas revela características importantes sobre a natureza dos dados de sepsis:

**1. Predominância de Distribuições Assimétricas:**
- A maioria das variáveis clínicas apresenta **assimetria positiva** (cauda à direita)
- Este padrão é **esperado em dados médicos**, onde valores extremos elevados são clinicamente significativos
- Variáveis como biomarcadores laboratoriais tendem a ter poucos casos com valores muito altos (pacientes críticos)

**2. Qualidade dos Dados:**
- Variabilidade significativa na **completude dos dados** entre diferentes variáveis
- Parâmetros de sinais vitais geralmente apresentam **maior completude** (coleta rotineira)
- Exames laboratoriais específicos podem ter **maior percentual de missing values** (coleta sob demanda)

**3. Implicações Clínicas das Distribuições:**
- **Distribuições normais** (fundo verde): Indicam parâmetros com comportamento fisiológico regular
- **Distribuições assimétricas** (fundo vermelho/amarelo): Sugerem presença de:
  - Valores de referência com limite inferior natural (ex: 0 para muitos biomarcadores)
  - Casos extremos clinicamente relevantes (pacientes em estado crítico)
  - Necessidade de transformações para modelagem

**4. Considerações para Modelagem:**
- Variáveis com **alta assimetria** podem beneficiar de transformações logarítmicas ou de Box-Cox
- A presença de **outliers** pode representar casos críticos clinicamente importantes
- **Diferentes escalas** entre variáveis requerem normalização/padronização
- **Missing values** precisam de estratégias de imputação específicas por tipo de variável

**5. Padrões por Categoria de Variável:**
- **Sinais Vitais**: Tendem a ter distribuições mais regulares, refletindo homeostase
- **Biomarcadores Laboratoriais**: Frequentemente assimétricos, com valores de referência bem definidos  
- **Parâmetros de Oxigenação**: Podem apresentar distribuições bimodais (pacientes ventilados vs. não-ventilados)
- **Variáveis Demográficas**: Distribuições específicas (ex: idade com padrão populacional)

## 4. Análise Bivariada

A análise bivariada examina as relações entre variáveis, focando especialmente na associação com sepsis e em correlações clinicamente relevantes.

### 4.1 Relacionando Variáveis com Alto Missing vs SepsisLabel

O intuito é identificar se alguma(s) variável(is) extremamente esparsa(s) possui(em) valor preditivo ou se podem ser descartadas.

In [ ]:
#Top 9 Variáveis com >90% Missing vs Sepsis


# Calcular % de missing values para todas as variáveis numéricas
missing_analysis = []
for col in X_train.select_dtypes(include=[np.number]).columns:
    missing_pct = (X_train[col].isnull().sum() / len(X_train)) * 100
    missing_analysis.append({'variavel': col, 'missing_pct': missing_pct})

missing_df = pd.DataFrame(missing_analysis).sort_values('missing_pct', ascending=False)

# Selecionar TOP 9 variáveis com >90% missing
vars_to_analyze = missing_df[missing_df['missing_pct'] > 90]['variavel'].head(9).tolist()

# Criar visualização estratégica: 3x3 grid para as 9 variáveis
fig, axes = plt.subplots(3, 3, figsize=(20, 16))
axes = axes.flatten()

for idx, var in enumerate(vars_to_analyze):  # Processa todas as 9 variáveis
    ax = axes[idx]
    
    # Dados válidos apenas
    temp_df = pd.DataFrame({
        'feature': X_train[var],
        'sepsis': y_train
    }).dropna()
    
    missing_pct = missing_df[missing_df['variavel'] == var]['missing_pct'].iloc[0]
    
    # Converter para labels
    temp_df['sepsis_label'] = temp_df['sepsis'].map({0: 'Não-Sepsis', 1: 'Sepsis'})
    
    # Boxplot principal
    sns.boxplot(data=temp_df, x='sepsis_label', y='feature', 
                palette=['#3498db', '#e74c3c'], ax=ax)
    
    # Calcular métricas de separabilidade (importantes para ML)
    no_sepsis_data = temp_df[temp_df['sepsis'] == 0]['feature']
    sepsis_data = temp_df[temp_df['sepsis'] == 1]['feature']
    
    # Métricas para avaliação de utilidade em ML
    median_diff = sepsis_data.median() - no_sepsis_data.median()
    separability = abs(median_diff) / no_sepsis_data.std() if no_sepsis_data.std() > 0 else 0
    data_loss = (1 - len(temp_df)/len(X_train)) * 100  # % dados perdidos
    
    # Decisão de ML baseada em critérios
    if separability > 0.3:
        decision = "MANTER"
        bg_color = '#f0fff0'  # Verde claro
    elif separability > 0.1:
        decision = "TRATAR"  
        bg_color = '#fff8f0'  # Laranja claro
    else:
        decision = "DESCARTAR"
        bg_color = '#fff0f0'  # Vermelho claro
    
    ax.set_facecolor(bg_color)
    
    # Título com métricas
    ax.set_title(f'{var} \nSep={separability:.2f} | Loss={data_loss:.1f}% | n={len(temp_df):,}', 
                fontsize=11, pad=10)
    ax.set_xlabel('Sepsis Status')
    ax.set_ylabel(var)
    
    # Stats box
    stats_text = f'Missing: {missing_pct:.0f}% | Separabilidade: {separability:.2f} | {decision}'
    ax.text(0.02, 0.98, stats_text, transform=ax.transAxes, 
            verticalalignment='top', fontsize=8,
            bbox=dict(boxstyle='round', facecolor='white', alpha=0.9))
    
    ax.grid(True, alpha=0.3)
            

plt.tight_layout()
plt.show()


Decisões:
- DESCARTAR: 8 variáveis
- TRATAR: 1 variáveis

Top Candidadatas por Separabilidade:
  - Fibrinogen                     | Sep: 0.134 | Miss: 99%
  - PTT                            | Sep: 0.085 | Miss: 97%
  - Bilirubin_total                | Sep: 0.077 | Miss: 99%

Aplicar Imputação Cuidadosa: Fibrinogen

Métricas Agregadas:
  - Separabilidade média: 0.055
  - Perda de dados média: 98.3%
  - ROI threshold calculado: 0.001

### 4.2 Emparelhando Variáveis Pouco Esparsas

A análise bivariada examina as relações entre duas variáveis, ajudando a identificar padrões, dependências e correlações que podem ser cruciais para o desenvolvimento de modelos preditivos.

In [ ]:
# Pair Plot para mostrar distribuições e relações entre variáveis
# Seleciona um subconjunto de variáveis para o pairplot (para performance)

# Calcular missing values para variáveis numéricas
numeric_cols = X_train.select_dtypes(include=[np.number]).columns
low_missing_cols = []

for col in numeric_cols:
    missing_pct = (X_train[col].isnull().sum() / len(X_train)) * 100
    if missing_pct < 20:  # Menos de 20% missing
        low_missing_cols.append(col)

print(f"Variáveis numéricas com <20% missing: {len(low_missing_cols)}")

# [1:6] -> Pula a variável Hour a fim de analisar variáveis diferentes
pairplot_vars = low_missing_cols[1:6] if len(low_missing_cols) >= 5 else low_missing_cols

print(f"Variáveis selecionadas para pairplot: {pairplot_vars}")

pairplot_data = X_train[pairplot_vars].copy()
pairplot_data['SepsisLabel'] = y_train

# Amostrar dados se o dataset for muito grande (para performance)
if len(pairplot_data) > 5000:
    pairplot_sample = pairplot_data.sample(n=5000, random_state=42)
    print(f"Usando amostra de 5000 registros para pairplot (de {len(pairplot_data)} total)")
else:
    pairplot_sample = pairplot_data
    print(f"Usando todos os {len(pairplot_data)} registros para pairplot")

# Remover linhas com valores faltantes para o pairplot
pairplot_sample = pairplot_sample.dropna()

print(f"\nCriando Pair Plot com {len(pairplot_sample)} amostras válidas...")
print(f"Variáveis incluídas: {pairplot_vars}")

# Configurar paleta de cores
sns.set_palette("Set1")

# Criar pairplot
plt.figure(figsize=(15, 12))

# Pairplot com diferenciação por classe
pair_grid = sns.pairplot(pairplot_sample, hue='SepsisLabel',
                        diag_kind='hist',
                        plot_kws={'alpha': 0.6},
                        diag_kws={'alpha': 0.7})

# Personalizar a legenda
pair_grid._legend.set_title("SepsisLabel")
for text, label in zip(pair_grid._legend.get_texts(), ['Não-Sepsis', 'Sepsis']):
    text.set_text(label)

plt.suptitle('Pair Plot - Análise de Relações entre Variáveis', y=1.02, fontsize=16)
plt.show()


## 5. Análise Multivariada

A análise multivariada examina as interações complexas entre múltiplas variáveis clínicas simultaneamente, identificando padrões de correlação e dependências que são cruciais para o entendimento fisiopatológico da sepsis.

In [ ]:
# Correlation Heatmap - Análise de correlações entre variáveis

# Definir variáveis numéricas com baixo missing (<20%) para análise de correlação
numeric_cols = X_train.select_dtypes(include=[np.number]).columns
low_missing_cols = []

for col in numeric_cols:
    missing_pct = (X_train[col].isnull().sum() / len(X_train)) * 100
    if missing_pct < 20:  # Menos de 20% missing
        low_missing_cols.append(col)

print(f"Variáveis numéricas com <20% missing: {len(low_missing_cols)}")
print(f"Lista: {low_missing_cols}")

# Criar subset dos dados para correlação
correlation_data = X_train[low_missing_cols].copy()

# Calcular matriz de correlação
correlation_matrix = correlation_data.corr()

print(f"MATRIZ DE CORRELAÇÃO")

# Criar heatmap de correlação
plt.figure(figsize=(16, 14))

# Configurar máscara para a diagonal superior (opcional)
mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))

# Criar heatmap
sns.heatmap(correlation_matrix,
            annot=True,           # Mostrar valores
            fmt='.2f',           # Formato dos números
            cmap='RdBu_r',       # Paleta de cores
            center=0,            # Centralizar em zero
            square=True,         # Células quadradas
            linewidths=0.5,      # Linhas entre células
            cbar_kws={"shrink": .8},  # Configurar colorbar
            mask=mask)           # Máscara para diagonal superior

plt.title('Matriz de Correlação - Heatmap', fontsize=16, pad=20)
plt.xlabel('Variáveis', fontsize=12)
plt.ylabel('Variáveis', fontsize=12)
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

# Encontrar correlações mais altas (excluindo diagonal)
correlation_matrix_no_diag = correlation_matrix.copy()
np.fill_diagonal(correlation_matrix_no_diag.values, np.nan)

# Achatar a matriz e ordenar por valor absoluto
corr_pairs = []
for i in range(len(correlation_matrix_no_diag)):
    for j in range(i+1, len(correlation_matrix_no_diag)):
        var1 = correlation_matrix_no_diag.index[i]
        var2 = correlation_matrix_no_diag.columns[j]
        corr_value = correlation_matrix_no_diag.iloc[i, j]
        if not np.isnan(corr_value):
            corr_pairs.append((var1, var2, corr_value))


## 6.Análise das Variáveis Categóricas

A identificação e análise das variáveis categóricas nos permite compreender as características demográficas e clínicas dos pacientes, bem como sua associação com o desenvolvimento de sepsis.

### 6.1 Visualização das Variáveis Categóricas

As visualizações nos permitem compreender melhor a distribuição das variáveis categóricas e sua associação com a ocorrência de sepsis de forma intuitiva.

In [ ]:
# Análise e visualização das variáveis categóricas: Unit1, Unit2 e Gender
# Criar barplots para distribuição e relação com sepsis

categorical_vars = ['Gender', 'Unit1', 'Unit2']

# Criar figura com subplots em grid 3x2 (3 linhas, 2 colunas)
fig, axes = plt.subplots(3, 2, figsize=(15, 18))

for idx, cat_var in enumerate(categorical_vars):
    # Preparar dados removendo valores faltantes
    temp_df = pd.DataFrame({
        'categorical': X_train[cat_var],
        'target': y_train
    }).dropna()
    
    # Gráfico 1: Distribuição da variável categórica (esquerda)
    ax1 = axes[idx, 0]
    
    # Contar valores únicos
    cat_counts = temp_df['categorical'].value_counts()
    
    # Criar gráfico de barras
    bars = ax1.bar(range(len(cat_counts)), cat_counts.values, 
                    color='#2c3e50', alpha=0.8, edgecolor='black')
    
    ax1.set_title(f'Distribuição de {cat_var}', fontsize=14, pad=15)
    ax1.set_xlabel(cat_var)
    ax1.set_ylabel('Frequência')
    ax1.set_xticks(range(len(cat_counts)))
    ax1.set_xticklabels(cat_counts.index, rotation=45)
    
    # Adicionar valores nas barras
    for bar, count in zip(bars, cat_counts.values):
        ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + max(cat_counts)*0.01,
                f'{count:,}', ha='center', va='bottom', fontweight='bold')
    
    # Gráfico 2: Variável categórica vs SepsisLabel (direita)
    ax2 = axes[idx, 1]
    
    # Preparar dados para stacked bar chart (números absolutos)
    crosstab_abs = pd.crosstab(temp_df['categorical'], temp_df['target'])
    
    # Preparar dados para percentuais (para os labels)
    crosstab_pct = pd.crosstab(temp_df['categorical'], temp_df['target'], normalize='index') * 100
    
    # Criar gráfico de barras empilhadas
    crosstab_abs.plot(kind='bar', stacked=True, ax=ax2, 
                        color=['#3498db', '#e74c3c'], alpha=0.8)
    
    ax2.set_title(f'{cat_var} vs SepsisLabel', fontsize=14, pad=15)
    ax2.set_xlabel(cat_var)
    ax2.set_ylabel('Frequência')
    ax2.set_xticklabels(crosstab_abs.index, rotation=45)
    ax2.legend(['Não-Sepsis', 'Sepsis'], title='SepsisLabel')
    
    # Adicionar percentual de sepsis acima das barras
    for i, category in enumerate(crosstab_abs.index):
        sepsis_pct = crosstab_pct.loc[category, 1]
        non_sepsis_count = crosstab_abs.loc[category, 0]
        sepsis_count = crosstab_abs.loc[category, 1]
        total_height = non_sepsis_count + sepsis_count
        
        max_total = crosstab_abs.sum(axis=1).max()  # Altura máxima das barras
        ax2.text(i, total_height + max_total*0.01, f'{sepsis_pct:.1f}%',
                ha='center', va='bottom', fontweight='bold', color='red')
        


plt.tight_layout()
plt.show()

### 6.2 Análise Verbosa com Inferência Estatística

Fazemos uma análise mais matemática envolvendo teste de hipóteses (Qui-quadrado e Valor-p) para complementar a interpretação do BarPlot e sustentar nossas decisões.

In [ ]:
for cat_var in categorical_vars:
    temp_df = pd.DataFrame({
        'categorical': X_train[cat_var],
        'target': y_train
    }).dropna()
    
    print(f"\n{cat_var.upper()}:")
    
    # Taxa de sepsis por categoria
    sepsis_rates = temp_df.groupby('categorical')['target'].agg(['count', 'sum', 'mean'])
    sepsis_rates.columns = ['Total', 'Sepsis_Count', 'Sepsis_Rate']
    sepsis_rates['Sepsis_Rate_%'] = sepsis_rates['Sepsis_Rate'] * 100
    
    print(sepsis_rates[['Total', 'Sepsis_Count', 'Sepsis_Rate_%']].round(2))
    
    # Teste qui-quadrado para independência
    from scipy.stats import chi2_contingency
    contingency_table = pd.crosstab(temp_df['categorical'], temp_df['target'])
    chi2, p_value, dof, expected = chi2_contingency(contingency_table)
    
    print(f"Chi-quadrado: {chi2:.4f} | P-valor: {p_value:.4f}")
    
    if p_value < 0.05:
        print("Associação estatisticamente significativa com sepsis")
    else:
        print("Não há associação estatisticamente significativa")